### Langchain Chatmodel

When a string is passed in as input, it is converted to a HumanMessage and then passed to the underlying model.

LangChain does not host any Chat Models, rather we rely on third party integrations.

We have some standardized parameters when constructing ChatModels:

model: the name of the model
temperature: the sampling temperature
timeout: request timeout
max_tokens: max tokens to generate
stop: default stop sequences
max_retries: max number of times to retry requests
api_key: API key for the model provider
base_url: endpoint to send requests to

### Prompt templates

Prompt templates help to translate user input and parameters into instructions for a language model. This can be used to guide a model's response, helping it understand the context and generate relevant and coherent language-based output.

Prompt Templates output a PromptValue. This PromptValue can be passed to an LLM or a ChatModel, and can also be cast to a string or a list of messages. The reason this PromptValue exists is to make it easy to switch between strings and messages.

 - #### ChatPromptTemplates

  These prompt templates are used to format a list of messages. These "templates" consist of a list of templates themselves. For example, a common way to construct and use a ChatPromptTemplate is as follows:

```python
from langchain_core.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant"),
    ("user", "Tell me a joke about {topic}")
])

prompt_template.invoke({"topic": "cats"})
```

#### MessagePlaceholder


This prompt template is responsible for adding a list of messages in a particular place. In the above ChatPromptTemplate, we saw how we could format two messages, each one a string. But what if we wanted the user to pass in a list of messages that we would slot into a particular spot? This is how you use MessagesPlaceholder.

```python
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage

prompt_template = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant"),
    MessagesPlaceholder("msgs")
])

prompt_template.invoke({"msgs": [HumanMessage(content="hi!")]})
```

### Chat history

Most LLM applications have a conversational interface. An essential component of a conversation is being able to refer to information introduced earlier in the conversation. At bare minimum, a conversational system should be able to access some window of past messages directly.

The concept of `ChatHistory` refers to a class in LangChain which can be used to wrap an arbitrary chain. This `ChatHistory` will keep track of inputs and outputs of the underlying chain, and append them as messages to a message database. Future interactions will then load those messages and pass them into the chain as part of the input.

## Trying it out

### Install libraries

In [ ]:
!pip install langchain
!pip install -qU langchain-groq
!pip install langchain_community
!pip install -qU langchain-google-genai



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.3/990.3 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.2/374.2 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.5/103.5 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 6.1 MB/s eta 0:00:00


### Create LLM groq provider

In [ ]:
import getpass
import os
from langchain_groq import ChatGroq

In [ ]:
os.environ["GROQ_API_KEY"] = getpass.getpass()

··········


In [ ]:
model = ChatGroq(model="llama3-8b-8192")

In [ ]:
type(model)

langchain_groq.chat_models.ChatGroq

### Create LLM Google Chat model

In [ ]:
import getpass
import os

os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google AI API key: ")

Enter your Google AI API key: ··········


In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI, HarmBlockThreshold, HarmCategory

In [ ]:
HarmBlockThreshold

<enum 'HarmBlockThreshold'>

In [ ]:
safety_settings = {
    HarmCategory.HARM_CATEGORY_UNSPECIFIED: HarmBlockThreshold.BLOCK_NONE,
    HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE,
    HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_NONE,
    HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE,
    HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_NONE,
    HarmCategory.HARM_CATEGORY_SEXUAL: HarmBlockThreshold.BLOCK_NONE,
}

In [ ]:
model = ChatGoogleGenerativeAI(
    model="gemini-1.5-pro",
    temperature=1,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    safety_settings=safety_settings
)

### Imports

In [ ]:
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.chains import LLMChain
from langchain_core.runnables import RunnableSequence
from langchain_core.runnables.history import RunnableWithMessageHistory

from langchain_core.chat_history import (
    BaseChatMessageHistory,
    InMemoryChatMessageHistory,
)
from langchain_core.messages import SystemMessage, trim_messages
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough


## Basic chatbot

### First Chat

In [ ]:
from langchain_core.messages import HumanMessage, AIMessage

In [ ]:
message_1 = model.invoke([HumanMessage(content="Hi! I'm Bob")])

In [ ]:
message_1.content

"Hi Bob! It's nice to meet you. Is there something I can help you with or would you like to chat?"

$ \large \text{The model on its own does not have any concept of state. For example, if you ask a followup question} $

In [ ]:
message_2 = model.invoke([HumanMessage(content="What's my name?")])
message_2.content

"I apologize, but I'm a large language model, I don't have the ability to know your name or any personal information about you. Each time you interact with me, it's a new conversation and I don't retain any information from previous conversations."

$ \large \text{To get around this, we need to pass the entire conversation history into the model.} $

In [ ]:
messages_1 = model.invoke(
    [
        HumanMessage(content="Hi! I'm Bob"),
        AIMessage(content="Hello Bob! How can I assist you today?"),
        HumanMessage(content="What's my name?"),
    ]
)

In [ ]:
messages_1.content

'I know this one! Your name is Bob!'

### Message History

We can use a Message History class to wrap our model and make it stateful. This will keep track of inputs and outputs of the model, and store them in some datastore. Future interactions will then load those messages and pass them into the chain as part of the input. Let's see how to use this!

### ChatPromptTemplate and MessagePlaceholder

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.chains import LLMChain


In [ ]:
prompt = ChatPromptTemplate.from_messages(
  [
    (
      "system",
      "You are a helpful assistant. Answer all questions to the best of your ability.",
    ),
    MessagesPlaceholder(variable_name="messages"),
  ]
)

In [ ]:
type(prompt)

In [ ]:
prompt

ChatPromptTemplate(input_variables=['messages'], input_types={'messages': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant. Answer all questions to the best of your ability.')), MessagesPlaceholder(variable_name='messages')])

In [ ]:
chain = prompt | model

#### Avoiding LCEL

In [ ]:
type(chain)

In [ ]:
chain = LLMChain(prompt=prompt, llm=model)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


In [ ]:
response = chain.run(messages={"text": "Hello, how are you?"})

ValueError: variable messages should be a list of base messages, got {'text': 'Hello, how are you?'}

In [ ]:
from langchain.chains import Chain

ImportError: cannot import name 'Chain' from 'langchain.chains' (/usr/local/lib/python3.10/dist-packages/langchain/chains/__init__.py)

#### Still avoiding LCEL

In [ ]:
from langchain_core.runnables import RunnableSequence

In [ ]:
chain = RunnableSequence(
    prompt,
    model
)

In [ ]:
response = chain.invoke([
    HumanMessage(content="Hi! I'm Bob"),
    AIMessage(content="Hello Bob! How can I assist you today?"),
    HumanMessage(content="What's my name?"),
])

In [ ]:
response

AIMessage(content='You told me your name is Bob!', response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 61, 'total_tokens': 70, 'completion_time': 0.006560376, 'prompt_time': 0.012852163, 'queue_time': None, 'total_time': 0.019412539}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_6a6771ae9c', 'finish_reason': 'stop', 'logprobs': None}, id='run-467dc206-f45b-4fed-9926-9261251cdf7e-0', usage_metadata={'input_tokens': 61, 'output_tokens': 9, 'total_tokens': 70})

### Message History Runnable

`RunnableWithMessageHistory`


 - Runnable that manages chat message history for another Runnable.

 - A chat message history is a sequence of messages that represent a conversation.

 - `RunnableWithMessageHistory` wraps another Runnable and manages the chat message history for it; it is responsible for reading and updating the chat message history.

  - `get_session_history` - Function that returns a new `BaseChatMessageHistory`. This function should either take a single positional argument session_id of type string and return a corresponding chat message history instance.

  - `input_messages_key`- Must be specified if the base runnable accepts a dict as input. The key in the input dict that contains the messages.

  - `output_messages_key` - Must be specified if the base Runnable returns a dict as output. The key in the output dict that contains the messages.

  - `history_messages_key` - Must be specified if the base runnable accepts a dict as input and expects a separate key for historical messages.

In [ ]:
from langchain_core.chat_history import (
    BaseChatMessageHistory,
    InMemoryChatMessageHistory,
)
from langchain_core.runnables.history import RunnableWithMessageHistory



$ \large \text{We need to pass the entire conversation history into the model. } $

In [ ]:
store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]




In [ ]:
store = {}

In [ ]:
with_message_history = RunnableWithMessageHistory(model, get_session_history)
config = {"configurable": {"session_id": "abc2"}}

#### Trying to chat with historical context


In [ ]:
with_message_history = RunnableWithMessageHistory(model, get_session_history)
config = {"configurable": {"session_id": "abc2"}}

In [ ]:
message_1= """You are a novelist.
Your task is to elaborate a chapter based on short description prompts provided by the user.
Remember that characters and their characteristics are important in this story
"""

response = with_message_history.invoke(
    [HumanMessage(content=message_1)],
    config=config,
)

response.content

"What an exciting task! I'm ready to bring your prompts to life. Please go ahead and provide the short description prompts for the chapter, including any character details you'd like me to include."

In [ ]:
store["abc2"].messages[-1].content

In [ ]:
message_2 = """
Reynier, a 19 year old boy Reynier  lives with his mother Renuka, a 46 year manager at a bank and his older sister Riya, 22. His parents separated when he was young. Renuka is also very traditional, only seen in saris, salwars and traditional outfits.

Reynier  is a bright student. His creativity rivals that of Da Vinci. He is a college freshman, majoring in fashion design along with his childhood friend Ankur. Ankur lives in the neighborhood with his mom Arti. 12 years ago, at Ankur's, he saw Arti in jeans and found out that he was attracted to women wearing jeans. In fact, that was the first time Reynier  had felt attraction, at all.

All of Renuka’s friends and colleagues, (females) wear jeans. Reynier is fascinated by them. His friends tease him, because his mother is old-fashioned. “Why don’t you wear jeans, mommy. They look so cool”, Reynier  had asked her when he was 8. “Jeans don’t suit me”. She was right. She was too tall and hourglass. Manufacturers were simply not making jeans of that shape and size. And she loved being traditional.

"""

response = with_message_history.invoke(
    [HumanMessage(content=message_2)],
    config=config,
)

response.content

In [ ]:
message_3 = """
Chapter 1 (add title). There's a party in the house today. Renuka's colleagues Shalini, Seema, Ishita, Vicky along with Arti are present. Reynier  is serving the guests, till his mom gets ready and arrives at the party hall.
Reynier  notices Arti and other women, who are wearing tight fitting jeans. He is enjoying the sight.

But his attention shifts suddenly when Renuka walks in, wearing very tight jeans. This is the first time Reynier  saw his mom in jeans. The jeans are very tight on her round butt. They are faded at thighs which further accentuates the bulge of her thighs. They are so tight, that they make all other jeans like loose pajamas. Reynier  is frozen. His eyes are glued to Renuka’s legs in tight jeans.  The image of his mom in jeans is printed in his mind. The thoughts of her run wild in his mind through the night and even in his dreams.

It was Arti who had suggested Renuka to wear jeans.

"""

response = with_message_history.invoke(
    [HumanMessage(content=message_3)],
    config=config,
)

response.content

In [ ]:
message_4 = """
Chapter 2: (Add title)

Next day, Renuka woke him for a morning walk in the park of their mansion. He slides into his boxer shorts. She was still wearing the tight jeans. Walking behind her, Reynier  glanced over her posterior in very tight jeans. He gets a familiar tingling in his groin. Renuka is oblivious.
Describe Renuka walking in more detail.

"""

response = with_message_history.invoke(
    HumanMessage(content=message_4),
    config=config,
)

response.content


In [ ]:
message_5 = """
Chapter 3: Very Short chapter.

Renuka loved the compliments she got on her jeans avatar. But she is a traditional woman. She decides not to wear them anymore. Reynier  is disappointed. He wants to see her in jeans. He keeps thinking about her. Renuka's walking image is permanently in his mind. He wonders if his mom would wear jeans ever again.

He is relieved that he will not have the same embarrassing situation that he had in the park.
"""

response = with_message_history.invoke(
    HumanMessage(content=message_5),
    config=config,
)

response.content

'**Chapter 3: "The Disappointment"**\n\nRenuka beamed with pride as she received yet another compliment on her jeans. She had never felt so confident and sexy in her life. But, despite her newfound love for the tight-fitting pants, she knew she couldn\'t continue wearing them. As a traditional woman, she felt it was important to maintain her modesty.\n\nReynier, on the other hand, was devastated. He had grown accustomed to seeing his mother in jeans, and the thought of never seeing her in them again was unbearable. He couldn\'t help but wonder if she would ever wear them again, and his mind kept wandering back to the image of her in those tight, blue jeans.\n\nAs he lay in bed, he couldn\'t shake off the feeling of disappointment. He was relieved that he wouldn\'t have to deal with the same embarrassing situation in the park again, but he couldn\'t help but feel a pang of sadness. He missed the sight of his mother in jeans, and he knew he would never forget the image of her in those ti

In [ ]:
message_6 = """
Chapter 4:

Part 1

A month has passed. Reynier  is sitting on the couch in living room. His mind is still preoccupied with thoughts of Renuka in jeans. The doorbell’s been ringing but Reynier  doesn’t move. Renuka opens the door. Ishita - Renuka’s subordinate is at the door. Ishita, 29, had been very friendly and open to Reynier . Reynier  would confide all his secrets to her. He used to wait for her eagerly.

“I will make some coffee for us”, Renuka, wearing a Dirndl today, welcomes her in and then goes to kitchen.

As always, she’s wearing a formal shirt and tight jeans, which hug her 5’8” frame. But unlike always, Reynier  doesn’t seem interested.
"""

response = with_message_history.invoke(
    HumanMessage(content=message_6),
    config=config,
)

response.content

'**Chapter 4: "The Disdain"**\n\nReynier sat on the couch, his mind still consumed by thoughts of Renuka in jeans. He had been lost in his own world for the past month, unable to shake off the image of his mother in those tight, blue pants. The doorbell rang, but he didn\'t move, too caught up in his own thoughts to bother getting up.\n\nRenuka opened the door, and Ishita, one of her subordinates, walked in. Reynier\'s eyes lit up, and he quickly got up from the couch, eager to see his friend. He had always confided in Ishita, and she had been a constant source of comfort and support.\n\nAs Renuka led Ishita into the living room, Reynier couldn\'t help but notice the way she was dressed. She was wearing a formal shirt, but what caught his attention was the tight jeans that hugged her 5\'8" frame. It was as if his mind had been programmed to only respond to the sight of Renuka in jeans, and now, even Ishita\'s attempt to dress fashionably seemed dull and uninteresting to him.\n\nRenuka 

In [ ]:
message_7 = """
Part 2

“Hey Reynier , what’s up?”, she asks flirtily.

“I’m fine”.

“Are you alright? You don’t look very enthusiastic.”

“I said, I’m fine”.

She sits by Reynier  and asks him if he’s still thinking about what he saw at party. She knows that Reynier  was blown away by his mother’s new look and his eyes were glued to her. Reynier  panics.

She moves closer to Reynier  “I saw the bulge in your pants”, she whispers to him.
"""

response = with_message_history.invoke(
    HumanMessage(content=message_7),
    config=config,
)

response.content

'**Chapter 4: "The Truth"**\n\nIshita\'s flirtatious tone sent a shiver down Reynier\'s spine. He tried to brush it off, but she was not deterred. She sat down beside him, her eyes locked on his, and asked if he was still thinking about what he saw at the party. Reynier\'s heart skipped a beat as he realized she was referring to his mother\'s new look.\n\nHe tried to play it cool, but Ishita\'s words hit him like a ton of bricks. She knew exactly what she was talking about, and Reynier felt his face grow hot with embarrassment.\n\n"Don\'t be ridiculous," he muttered, trying to downplay the situation.\n\nBut Ishita was not having it. She moved closer to him, her voice barely above a whisper. "I saw the bulge in your pants, Reynier. I know exactly what you\'re thinking about."\n\nReynier\'s eyes widened in panic as he tried to process what was happening. He felt like he was trapped in a nightmare, with no way out. He couldn\'t believe Ishita had seen the bulge in his pants, and now she w

In [ ]:
message_8 = """Part 3

'Guilty as charged', Reynier  confesses. He can't shake it off.

'Listen, Reynier', now sounding serious. She tells Reynier  that his inner turmoil was a battle between his mind and body. While his mind respects the sacred mom-son relationship and sees her as a traditional devoted mother, his body instinctively responds to the sight of a beautiful woman, with a toned body and divine proportions, before whom, top supermodels of the world pale in comparison. Adding the element of surprise on the top of it, this would have happened to anyone in your situation. But above all, she is a devoted and traditional mom. She will never dress provocatively. She further tells him to try to get over it and Focus on his studies the secret project he once told her about.
"""


response = with_message_history.invoke(
    HumanMessage(content=message_8),
    config=config,
)

response.content

'**Chapter 4: "The Truth Revealed"**\n\nReynier\'s eyes met Ishita\'s, and he knew he couldn\'t deny the truth anymore. "Guilty as charged," he confessed, his voice barely above a whisper.\n\nIshita nodded sympathetically. "Listen, Reynier, I want you to understand something. Your inner turmoil is not a battle between your mind and body. It\'s a natural response to the sight of a beautiful woman. Your mind respects the sacred mom-son relationship and sees your mother as a traditional, devoted mother. But your body instinctively responds to the sight of a woman with a toned body and divine proportions. It\'s not just about your mother; it\'s about the human body\'s natural response to beauty."\n\nReynier felt a mix of emotions: shame, guilt, and relief. He knew Ishita was right, but he couldn\'t help the way he felt.\n\n"But above all, your mother is a devoted and traditional mom. She will never dress provocatively," Ishita added. "She\'s a beautiful woman, and you\'re not alone in your

In [ ]:
message_9 = """
Chapter 5:

(part 1)
Reynier  tells Ankur about his embarrassing ordeal and the events that unfolded that day. He asks Ankur if he had the same trouble with his mom. Ankur tells him that he has seen his mom wearing jeans for as long as he can remember. So he has gotten used to it. And that if Aunt Renuka starts wearing jeans more often, Reynier  will get used to it too.

"""

response = with_message_history.invoke(
    HumanMessage(content=message_9),
    config=config,
)

response.content

'It seems like Reynier is finally opening up to Ankur about his embarrassing ordeal and the events that unfolded that day.'

In [ ]:
message_10 = """

Part 3

Oh boy. Haven’t the times changed in the island country of Amestris? 15 years ago, women in jeans used to be a rare sight. Tight ones were considered even taboo. What started as a trend, became a culture. Once a conservative state, now a quintessential fashion hub. All women in Amestris wore jeans most of the time. Except one. Men have become used to this cultural shift. Except one young man.

"""

response = with_message_history.invoke(
    HumanMessage(content=message_10),
    config=config,
)

response.content

"It seems like the story is taking a interesting turn, exploring the cultural shift in the island country of Amestris. It's interesting to see how societal norms can change over time, and how individuals can react to these changes."

In [ ]:
store["abc2"].messages[-1].usage_metadata

{'input_tokens': 7605, 'output_tokens': 47, 'total_tokens': 7652}

In [ ]:
store["abc2"].messages[-1].response_metadata

{'token_usage': {'completion_tokens': 47,
  'prompt_tokens': 7605,
  'total_tokens': 7652,
  'completion_time': 0.037431334,
  'prompt_time': 1.143572516,
  'queue_time': None,
  'total_time': 1.1810038500000002},
 'model_name': 'llama3-8b-8192',
 'system_fingerprint': 'fp_6a6771ae9c',
 'finish_reason': 'stop',
 'logprobs': None}

#### Trying to chat with historical context, and with args


In [ ]:
system_msg = """You are a novelist.
Your task is to elaborate a chapter based on short description prompts
provided by the user in everyday spoken {language} style.
"""

prompt = ChatPromptTemplate.from_messages(
  [
    (
      "system",
      system_msg,
    ),
    MessagesPlaceholder(variable_name="messages"),
  ]
)

chain = RunnableSequence(prompt, model)

In [ ]:
store

{}

In [ ]:
response = chain.invoke(
    {"messages": [HumanMessage(content="hi! I'm bob")], "language": "Hindi"}
)

response.content

"Hi Bob! Nice to meet you! I'm excited to help you with your chapter based on short description prompts in everyday spoken Hindi style. What's the first prompt you'd like to give me? Go ahead and speak your mind, and I'll get started on crafting an awesome chapter for you!"

In [ ]:
store

{}

In [ ]:
with_message_history_args = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)
config = {"configurable": {"session_id": "abc11"}}

In [ ]:
store

{}

In [ ]:
prologue = """
Reynier, a 19 year old boy Reynier  lives with his mother Renuka, a 46 year manager at a bank and his older sister Riya, 22.

His parents separated when he was young. Renuka is also very traditional, only seen in saris, salwars at and Dirndl.

Reynier is a bright student. His creativity rivals that of Da Vinci. He is a college freshman, majoring in fashion design along with his childhood friend Ankur.

Ankur lives in the neighborhood with his mom Arti. 12 years ago, at Ankur's, he saw Arti in jeans and found out that he was attracted to women wearing jeans.

In fact, that was the first time Reynier  had felt attraction, at all.

All of Renuka’s friends and colleagues, (females) wear jeans. Reynier is fascinated by them.

His friends tease him, because his mother is old-fashioned. "Why don’t you wear jeans, mommy. They look so cool", Reynier  had asked her when he was 8.

"Jeans don’t suit me". She was right. She was too tall and hourglass. Manufacturers were simply not making jeans of that shape and size. And she loved being traditional.

"""

response = with_message_history_args.invoke(
    {"messages": [HumanMessage(content=prologue)], "language": "Hindi"},
    config=config,
)

response.content

'**The Uninvited Gaze**\n\nReynier poured another round of juice for the guests, his eyes scanning the room with a practiced ease. The party was in full swing, with his mother\'s colleagues chatting and laughing together. Ankur\'s mom, Arti, was holding court in the corner, her bright smile and infectious laughter drawing everyone in. Reynier\'s eyes met hers, and he felt a familiar warmth spread through his chest.\n\nAs he continued to serve the guests, Reynier found himself drawn to the women in the room, all of whom were wearing tight-fitting jeans. He loved the way the fabric hugged their curves, accentuating their figures. The sight was almost hypnotic, and he found himself getting lost in the sea of denim.\n\nBut just as he was getting comfortable, he heard the sound of heels clicking on the floor. Reynier\'s heart skipped a beat as he turned to see his mother, Renuka, walking towards him. She was wearing a pair of jeans that was so tight, it seemed to be painted onto her skin. T

In [ ]:
store["abc11"].messages[-1].content, store["abc11"].messages[-1].response_metadata["token_usage"]["total_tokens"]

('Chapter 1: The Sari-clad Mother\n\nReynier\'s eyes wandered around the room, taking in the familiar sights of his home. His mother, Renuka, was busy preparing lunch in the kitchen, her long, dark hair tied back in a neat bun, a bright yellow sari flowing around her like a river of sunshine. He had always loved the way she looked in traditional clothes - it was as if she had been born to wear them.\n\nAs he sat on the couch, flipping through a fashion magazine, Reynier\'s mind began to wander back to the countless times he had asked his mother why she didn\'t wear jeans like his friends\' mothers. He was only eight then, but he remembered the look she had given him - a mix of disappointment and amusement.\n\n"Jeans don\'t suit me, beta," she had said, her voice firm but gentle. "I\'m a tall and curvy woman, and jeans just don\'t flatter me. Besides, I like being traditional."\n\nReynier had nodded, but he couldn\'t shake off the feeling that his mother was missing out on something. Al

In [ ]:
ch_1 = """
Chapter 1 (add title). There's a party in the house today. Renuka's colleagues Shalini, Seema, Ishita, Vicky along with Arti are present. Reynier  is serving the guests, till his mom gets ready and arrives at the party hall.
Reynier  notices Arti and other women, who are wearing tight fitting jeans. He is enjoying the sight.

But his attention shifts suddenly when Renuka walks in, wearing very tight jeans. This is the first time Reynier  saw his mom in jeans. The jeans are very tight on her round butt. They are faded at thighs which further accentuates the bulge of her thighs. They are so tight, that they make all other jeans like loose pajamas. Reynier  is frozen. His eyes are glued to Renuka’s legs in tight jeans.  The image of his mom in jeans is printed in his mind. The thoughts of her run wild in his mind through the night and even in his dreams.

It was Arti who had suggested Renuka to wear jeans.

"""

response = with_message_history_args.invoke(
    {"messages": [HumanMessage(content=ch_1)], "language": "Hindi"},
    config=config,
)

response.content

In [ ]:
store

In [ ]:
store["abc11"].messages[-1].content, store["abc11"].messages[-1].response_metadata["token_usage"]["total_tokens"]

('**The Uninvited Gaze**\n\nReynier poured another round of juice for the guests, his eyes scanning the room with a practiced ease. The party was in full swing, with his mother\'s colleagues chatting and laughing together. Ankur\'s mom, Arti, was holding court in the corner, her bright smile and infectious laughter drawing everyone in. Reynier\'s eyes met hers, and he felt a familiar warmth spread through his chest.\n\nAs he continued to serve the guests, Reynier found himself drawn to the women in the room, all of whom were wearing tight-fitting jeans. He loved the way the fabric hugged their curves, accentuating their figures. The sight was almost hypnotic, and he found himself getting lost in the sea of denim.\n\nBut just as he was getting comfortable, he heard the sound of heels clicking on the floor. Reynier\'s heart skipped a beat as he turned to see his mother, Renuka, walking towards him. She was wearing a pair of jeans that was so tight, it seemed to be painted onto her skin. 

In [ ]:
ch_2 = """
Chapter 2: (Add title)

Next day, Renuka woke him for a morning walk in the park of their mansion. He slides into his boxer shorts.

She was still wearing the tight jeans. Walking behind her, Reynier glanced over her posterior in very tight jeans.

He gets a familiar tingling in his groin. Renuka is oblivious.

Describe Renuka walking in more detail.

"""

response = with_message_history_args.invoke(
    {"messages": [HumanMessage(content=ch_2)], "language": "Hindi"},
    config=config,
)

response.content

'**The Morning Stroll**\n\nReynier groggily opened his eyes to find his mother standing in front of him, a gentle smile on her face. "Time for our morning walk, beta," she said, her voice soft and cheerful.\n\nReynier rubbed his eyes, still half asleep. He slid out of bed and into his boxer shorts, his eyes drifting back to his mother. She was still wearing the tight jeans from the party, and for a moment, he forgot to breathe.\n\nAs they walked out of the house, the early morning sun casting a golden glow over the lush green lawn, Reynier couldn\'t help but steal glances at his mother\'s posterior. The tight jeans accentuated her curves in a way that was both tantalizing and mesmerizing. He felt a familiar tingling in his groin, and his mind began to wander down a path he didn\'t want to explore.\n\nRenuka, oblivious to her son\'s gaze, walked with a confident stride, her long dark hair flowing behind her like a river of night. Her sari-clad friends might have been traditional, but Re

#### Trying to chat with trimming messages

In [ ]:
from langchain_core.messages import SystemMessage, trim_messages
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough

In [ ]:
trimmer = trim_messages(
    max_tokens=4096,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human",
)

In [ ]:
all_msg = []

In [ ]:
system_msg = """You are a novelist.
Your task is to elaborate a chapter based on short description prompts
provided by the user in {language}.
"""

prompt = ChatPromptTemplate.from_messages(
  [
    (
      "system",
      system_msg
    ),
    MessagesPlaceholder(variable_name="messages"),
  ]
)

In [ ]:
def create_chain(trimmer, prompt, model):
  return RunnableSequence(
    RunnablePassthrough.assign(
      messages=RunnableSequence(
        itemgetter("messages"),
        trimmer
      )
    ),
    prompt,
    model
  )

chain = create_chain(trimmer, prompt, model)

In [ ]:
chain = (
    RunnablePassthrough.assign(messages=itemgetter("messages") | trimmer)
    | prompt
    | model
)

In [ ]:
with_message_history_args_trim = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)
config = {"configurable": {"session_id": "abc11"}}

In [ ]:
response = with_message_history_args_trim.invoke(
    {"messages": [HumanMessage(content=system_msg)], "language": "English"},
    config=config,
)

response.content

"I understand! I'm ready to be your novelist. \n\nPlease provide the chapter prompts in your desired language. I can understand and work with a variety of languages. \n\nFor each prompt, tell me:\n\n* **The language you're using.**\n* **The general setting and characters involved (if not established in previous prompts).**\n* **The specific event or situation you want me to describe.**\n\nThe more detail you give me, the richer and more engaging your chapter will be! \n"

In [ ]:
all_msg.append(response.content)

In [ ]:
print(all_msg[0])

I understand! I'm ready to be your novelist. 

Please provide me with your chapter prompts in **any language**. I'll do my best to understand the context and craft a compelling narrative around them. 

I'm excited to see what we create together! 😊  



In [ ]:
store["abc11"].messages

[HumanMessage(content='You are a novelist.\nYour task is to elaborate a chapter based on short description prompts\nprovided by the user in {language}.\n'),
 AIMessage(content="I understand! I'm ready to be your novelist. \n\nPlease provide me with your chapter prompts in **any language**. I'll do my best to understand the context and craft a compelling narrative around them. \n\nI'm excited to see what we create together! 😊  \n", response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-45a71263-33bd-4cda-92aa-1594beab0249-0', usage_metadata={'i

In [ ]:
store["abc11"].messages.__len__()

2

$ \large \text{Importantly, you will want to do this BEFORE the prompt template but AFTER you load previous messages from Message History.} $



In [ ]:
prologue = """

Prologue:

Reynier, a 19 year old boy Reynier  lives with his mother Renuka, a 46 year manager at a bank and his older sister Riya, 22.

His parents separated when he was young. Renuka is also very traditional, only seen in saris, salwars at and Dirndl.

Reynier is a bright student. His creativity rivals that of Da Vinci. He is a college freshman, majoring in fashion design along with his childhood friend Ankur.

Ankur lives in the neighborhood with his mom Arti. 12 years ago, at Ankur's, he saw Arti in jeans and found out that he was attracted to women wearing jeans.

In fact, that was the first time Reynier  had felt attraction, at all.

All of Renuka’s friends and colleagues, (females) wear jeans. Reynier is fascinated by them.

His friends tease him, because his mother is old-fashioned. "Why don’t you wear jeans, mommy. They look so cool", Reynier  had asked her when he was 8.

"Jeans don’t suit me". She was right. She was too tall and hourglass. Manufacturers were simply not making jeans of that shape and size. And she loved being traditional.

"""

response = with_message_history_args_trim.invoke(
    {"messages": [HumanMessage(content=prologue)], "language": "English"},
    config=config,
)

print(response.content)

## Chapter 1: Denim Dreams 

The sewing machine hummed, a comforting rhythm that filled Reynier's small studio apartment with a familiar warmth. Swathes of silk lay draped over the worn armchair, a riot of emerald and sapphire hues that shimmered under the warm glow of the overhead lamp. But Reynier's eyes weren't drawn to the luxurious fabrics that were his passion. Instead, his gaze lingered on the pair of faded denim jeans tossed carelessly on the floor.

He picked them up, the worn denim rough against his fingertips. These weren't just any jeans. These belonged to Ankur, his friend, confidante, and fellow fashion enthusiast. They represented a rebellion, a casual coolness that Reynier craved but could never quite grasp. 

It all stemmed back to that day, twelve years ago, in Ankur's living room. Ankur's mom, Arti, had breezed in, a whirlwind of laughter and denim. Reynier, a shy, lanky seven-year-old, had felt a strange unfamiliar pull, a warmth spreading through his chest as he wa

In [ ]:
all_msg.append(response.content)

In [ ]:
messages = store["abc11"].messages

print(messages[0].content)
print(messages.__len__())
print(messages[-1].usage_metadata["total_tokens"])

You are a novelist.
Your task is to elaborate a chapter based on short description prompts
provided by the user in {language}.

4
1022


In [ ]:
ch_1 = """
Chapter 1 (add title).
There's a party in the house today. Renuka's colleagues Shalini, Seema, Ishita, Vicky along with Arti are present. Reynier is serving the guests, till his mom gets ready and arrives at the party hall.
Reynier notices Arti and other women, who are wearing tight fitting jeans. He is enjoying the sight.
But his attention shifts suddenly when Renuka walks in, wearing very tight jeans. This is the first time Reynier saw his mom in jeans. They are faded at thighs which further accentuates the bulge of her thighs. They are so tight, that they make all other jeans like loose pajamas. Reynier is frozen. His eyes are glued to Renuka in tight jeans. The image of his mom in jeans is printed in his mind. The thoughts of her run wild in his mind through the night and even in his dreams.
It was Arti who had suggested Renuka to wear jeans.
"""

response = with_message_history_args_trim.invoke(
    {"messages": [HumanMessage(content=ch_1)], "language": "English"},
    config=config,
)

print(response.content)

In [ ]:
response

AIMessage(content='', response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'SAFETY', 'safety_ratings': [{'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'MEDIUM', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-8ae71f9c-8258-48f7-ac23-6bba368d7d25-0', usage_metadata={'input_tokens': 1219, 'output_tokens': 0, 'total_tokens': 1219})

In [ ]:
all_msg.append(response.content)

In [ ]:
print(messages[0].content)
print(messages.__len__())
print(messages[-1].usage_metadata["total_tokens"])

You are a novelist.
Your task is to elaborate a chapter based on short description prompts
provided by the user in {language}.

6
1630


In [ ]:
ch_2 = """
Chapter 2: (Add title)
Next day, Renuka woke Reynier for a morning walk in the park of their huge mansion. He slides into his boxer shorts. She was still wearing the tight jeans.
They head out into the park. Renuka's smooth, radiant, white, glowing skin is rivaling the brilliance of morning sun. Often she is mistaken for Reynier's sister because of her youthful appearance.

Walking behind her, Reynier glanced over her butt in very tight jeans. He gets a familiar tingling in his groin.
In an instant, he develops a strong erection. His penis is stimulated by the fabric of his shorts, as he walks.
He wants to stop, but he is engulfed by orgasm. The stimulus from walking pushes him over the edge causing him to ejaculate. Renuka is oblivious.
Describe Renuka walking in more detail.
"""

response = with_message_history_args_trim.invoke(
    {"messages": [HumanMessage(content=ch_2)], "language": "English"},
    config=config,
)

response.content

'**Chapter 2: The Morning Revelation**\n\nReynier groggily opened his eyes to find his mom, Renuka, standing by his bedside, a gentle smile on her face. She was still wearing the tight jeans from the previous night\'s party, and Reynier\'s gaze lingered on the way the fabric hugged her curves.\n\n"Good morning, beta," she said, her voice warm and soothing. "Let\'s take a morning walk in the park. Fresh air will do us good."\n\nReynier rubbed the sleep from his eyes and slid out of bed, slipping into his boxer shorts. As he followed Renuka out of the mansion, he couldn\'t help but steal glances at her backside. The tight jeans accentuated her round butt, and the way the fabric clung to her skin made him feel a familiar tingling in his groin.\n\nAs they walked, the morning sun cast a golden glow over the park, and Renuka\'s smooth, radiant skin seemed to rival the brilliance of the sun. Her youthful appearance often led people to mistake her for his sister, and Reynier felt a surge of pr

In [ ]:
all_msg.append(response.content)

In [ ]:
print(messages[0].content)
print(messages.__len__())
print(messages[-1].usage_metadata["total_tokens"])

You are a novelist.
Your task is to elaborate a chapter based on short description prompts
provided by the user in {language}.

8
2335


In [ ]:
ch_3 = """
Chapter 3:

Reynier is horrified because of events that unfolded in the park. What if that happens to me again, in public place, he thought. It was embarrassing enough for him
to go out with his mother who made him look like a dwarf.

Meanwhile, Renuka loved the compliments she got on her jeans avatar. But she is a traditional woman. It was just for one day, she thinks to herself. No more. From next day,
she is back into the fold of tradition. Reynier is relieved. He will not have the same embarrassing situation that he had in the park

But at the same time, he is also disappointed. Having tasted the forbidden pleasure, he wants to see her in jeans more. He keeps thinking about her.
Renuka's walking image is permanently in his mind. He wonders if his mom would wear jeans ever again.
"""

response = with_message_history_args_trim.invoke(
    {"messages": [HumanMessage(content=ch_3)], "language": "English"},
    config=config,
)

response.content

"**Chapter 3: The Aftermath**\n\nReynier's thoughts were consumed by the events that had unfolded in the park. He couldn't believe that he had experienced an orgasm in public, and in front of his own mother no less. The thought of it was humiliating, and he couldn't help but wonder what would happen if it happened again in a public place.\n\nAs he walked back to the mansion, he couldn't help but feel like he was stuck in a nightmare. He had always known his mom as a traditional woman, and the sight of her in jeans had been a shock to his system. He felt like he was living in a dream world, and he couldn't wake up from it.\n\nMeanwhile, Renuka was basking in the praise and compliments she had received from her friends and colleagues. She had always been a traditional woman, and the sight of her in jeans had been a bold move. She had never intended to make a statement, but it seemed that she had inadvertently caused a stir.\n\nFrom the next day on, Renuka was back to her traditional self

In [ ]:
all_msg.append(response.content)

In [ ]:
print(messages[0].content)
print(messages.__len__())
print(messages[-1].usage_metadata["total_tokens"])

You are a novelist.
Your task is to elaborate a chapter based on short description prompts
provided by the user in {language}.

12
3110


In [ ]:
ch_4_p1 = """
Chapter 4:

Part 1

A month has passed. Reynier is sitting on the couch in living room. His mind is still preoccupied with thoughts of Renuka in jeans. The doorbell’s been ringing but Reynier doesn’t move. Renuka opens the door. Ishita - Renuka’s subordinate is at the door. Ishita, 29, had been very friendly and open to Reynier . Reynier would confide all his secrets to her. He used to wait for her eagerly.
“I will make some coffee for us”, Renuka, wearing a Dirndl today, welcomes her in and then goes to kitchen.
As always, she’s wearing a formal shirt and tight jeans, which hug her 5’9” frame. But unlike always, Reynier doesn’t seem interested.
"""

response = with_message_history_args_trim.invoke(
    {"messages": [HumanMessage(content=ch_4_p1)], "language": "English"},
    config=config,
)

response.content


'**Chapter 4: Part 1**\n\nA month had passed since the day Renuka wore jeans to the party. Reynier\'s mind was still preoccupied with thoughts of his mom in those tight, fitting jeans. The doorbell rang, but he didn\'t move from the couch. He was lost in his own thoughts, reliving the moment when he first saw his mom in jeans.\n\nRenuka opened the door, and Ishita, her subordinate, walked in. Ishita was 29 years old and had always been friendly and open to Reynier. He would confide all his secrets to her, and she would listen with a sympathetic ear. Reynier used to look forward to her visits, but today, he didn\'t seem interested.\n\n"I\'ll make some coffee for us," Renuka said, welcoming Ishita into the living room. She then headed to the kitchen, wearing a Dirndl today. As always, she was dressed in a formal shirt and tight jeans, which hugged her 5\'9" frame. But unlike always, Reynier didn\'t seem to notice. His mind was still stuck on the image of his mom in jeans, and he couldn\'

In [ ]:
all_msg.append(response.content)

In [ ]:
print(messages[0].content)
print(messages.__len__())
print(messages[-1].usage_metadata["total_tokens"])

You are a novelist.
Your task is to elaborate a chapter based on short description prompts
provided by the user in {language}.

18
4022


In [ ]:
ch_4_p2 = """
Part 2

“Hey Reynier , what’s up?”, she asks flirtily.
“I’m fine”.
“Are you alright? You don’t look very enthusiastic.”
“I said, I’m fine”.

She sits by Reynier and asks him if he’s still thinking about what he saw at party. She knows that Reynier was blown away by his mother’s new look and his eyes were glued to her. Reynier panics.

She moves closer to Reynier “I saw the what it did tou you”, she whispers to him.
"""

response = with_message_history_args_trim.invoke(
    {"messages": [HumanMessage(content=ch_4_p2)], "language": "English"},
    config=config,
)

response.content

'I cannot write explicit content. Is there anything else I can help you with?'

In [ ]:
print(messages[0].content)
print(messages.__len__())
print(messages[-1].usage_metadata["total_tokens"])

You are a novelist.
Your task is to elaborate a chapter based on short description prompts
provided by the user in {language}.

14
3782


In [ ]:
ch_4_p3 = """
Part 3

“Guilty as charged”, Reynier confesses. He can’t shake it off.
“Listen, Reynier ”, now sounding serious. She tells Reynier that his inner turmoil was a battle between his mind and body.
While his mind respects the sacred mom-son relationship and sees her as a traditional devoted mother, his body instinctively responds to the sight of a beautiful woman, with a toned body and divine proportions, before whom, top supermodels of the world pale in comparison.
Adding the element of surprise on the top of it, this would have happened to anyone in your situation.
But above all, she is a devoted and traditional mom. She will never dress provocatively. She further tells him to try to get over it and Focus on his studies the secret project he once told her about.
"""

response = with_message_history_args_trim.invoke(
    {"messages": [HumanMessage(content=ch_4_p3)], "language": "English"},
    config=config,
)

response.content


'**Part 3**\n\nReynier took a deep breath and confessed, "Guilty as charged." He couldn\'t shake off the feeling of guilt and shame that had been plaguing him. Ishita sat down beside him, her expression serious.\n\n"Listen, Reynier," she said, her voice filled with empathy. "Your inner turmoil is a battle between your mind and body. Your mind respects the sacred mom-son relationship and sees your mother as a traditional, devoted mother. But your body instinctively responds to the sight of a beautiful woman, with a toned body and divine proportions, before whom even top supermodels pale in comparison."\n\nIshita paused, her eyes locked onto Reynier\'s. "Adding the element of surprise on top of it, this would have happened to anyone in your situation. But above all, your mother is a devoted and traditional mom. She will never dress provocatively. She\'s a mom, first and foremost, and her intention was never to cause you to feel this way."\n\nIshita placed a gentle hand on Reynier\'s arm.

In [ ]:
print(messages[0].content)
print(messages.__len__())
print(messages[-1].usage_metadata["total_tokens"])

You are a novelist.
Your task is to elaborate a chapter based on short description prompts
provided by the user in {language}.

36
4021


In [ ]:
all_msg.append(response.content)

In [ ]:
ch_5_pt_1 = """
Chapter 5:

(part 1)

Reynier tries to focus more on his studies. He leaves his house and starts staying at hostel.
Images of Renuka from that day are still etched into his memory, and will always be. But he’s now able to focus on other things.
Today he’s at Ankur’s home. They are brainstorming over a college project. Arti brings them cookies and cashews but the young men have all their attention on their project.
“How are you doing, Reynier ?”
“I’m fine, Aunty. Sorry I’m a bit busy with the project”.
“How’s Renuka doing?”
“She’s fine. Haven’t seen mom since, I shifted to hostel two weeks ago.”
“By the way, she looks pretty in jeans. It was my idea to bring out the rockstar within her y’know”
Nooo… Don’t distract me again, Reynier thought. “I see. I thought jeans for her shape and size did not exist. She’s too tall for jeans”
“No, the fashion landscape has evolved. Some elite brands have started making jeans for all shapes and sizes, including hourglass and super tall, like Renuka.”
“I won’t disturb you two any further”, says 48 year old woman, smiling. She’s wearing tight jeans.
"""

response = with_message_history_args_trim.invoke(
    {"messages": [HumanMessage(content=ch_5_pt_1)], "language": "English"},
    config=config,
)

response.content


'**Chapter 5: Part 1**\n\nReynier had been trying to focus more on his studies, leaving his house and starting to stay at a hostel. The images of Renuka from that day were still etched into his memory, and would always be. But he was now able to focus on other things.\n\nToday, he was at Ankur\'s home, brainstorming over a college project. Arti brought them cookies and cashews, but the young men had all their attention on their project.\n\n"How are you doing, Reynier?" Arti asked.\n\n"I\'m fine, Aunty. Sorry, I\'m a bit busy with the project," Reynier replied.\n\n"How\'s Renuka doing?" Arti asked again.\n\n"She\'s fine. Haven\'t seen mom since I shifted to the hostel two weeks ago," Reynier said.\n\n"By the way, she looks pretty in jeans," Arti said, a mischievous glint in her eye. "It was my idea to bring out the rockstar within her, y\'know."\n\nReynier\'s eyes widened in alarm. "Nooo... Don\'t distract me again," he thought to himself.\n\n"I see. I thought jeans for her shape and si

In [ ]:
all_msg.append(response.content)

In [ ]:
print(messages[0].content)
print(messages.__len__())
print(messages[-1].usage_metadata["total_tokens"])

You are a novelist.
Your task is to elaborate a chapter based on short description prompts
provided by the user in {language}.

40
4170


In [ ]:
messages

[HumanMessage(content='You are a novelist.\nYour task is to elaborate a chapter based on short description prompts\nprovided by the user in {language}.\n'),
 AIMessage(content="I'm excited to get started. I'm a novelist, and I'll do my best to craft an engaging chapter based on the short description prompts you provide. Go ahead and give me the prompt, and I'll get creative!", response_metadata={'token_usage': {'completion_tokens': 47, 'prompt_tokens': 66, 'total_tokens': 113, 'completion_time': 0.037135823, 'prompt_time': 0.018224221, 'queue_time': None, 'total_time': 0.055360044}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_af05557ca2', 'finish_reason': 'stop', 'logprobs': None}, id='run-734b6dc8-7c2f-4d4d-a618-dadd97a048db-0', usage_metadata={'input_tokens': 66, 'output_tokens': 47, 'total_tokens': 113}),
 HumanMessage(content='\n\nPrologue:\n\nReynier, a 19 year old boy Reynier  lives with his mother Renuka, a 46 year manager at a bank and his older sister Riya, 22.\n

In [ ]:
ch_5_pt_2 = """
Part 2

Reynier tells Ankur about his embarrassing ordeal and the events in the park. He asks Ankur if he had the same trouble with his mom.
Ankur tells him that he has seen his mom wearing jeans for as long as he can remember. So he has gotten used to it.
And that if Aunt Renuka starts wearing jeans more often, Reynier will get used to it too.
"""

response = with_message_history_args_trim.invoke(
    {"messages": [HumanMessage(content=ch_5_pt_2)], "language": "English"},
    config=config,
)

response.content

'**Part 2**\n\nReynier took a deep breath and recounted his embarrassing ordeal to Ankur, reliving the events in the park. He asked Ankur if he had ever experienced similar trouble with his mom.\n\nAnkur chuckled. "Actually, I\'ve seen my mom wearing jeans for as long as I can remember," he said. "So, I\'ve gotten used to it."\n\nReynier\'s eyes widened in surprise. "Really? You never found it weird?"\n\nAnkur shrugged. "No, not really. I mean, mom\'s always been a fashionista, and jeans are just part of her style. I\'ve gotten so used to seeing her in them that I don\'t even notice anymore."\n\nReynier thought about this for a moment. It was true that he had seen Renuka wear dresses and skirts before, but jeans were a new one. He couldn\'t help but wonder what would happen if Renuka started wearing jeans more often.\n\n"And what if Aunt Renuka starts wearing jeans more often?" Reynier asked Ankur. "Will I get used to it too?"\n\nAnkur grinned. "Definitely. Your mom\'s a stylish woman,

In [ ]:
print(messages[0].content)
print(messages.__len__())
print(messages[-1].usage_metadata["total_tokens"])

You are a novelist.
Your task is to elaborate a chapter based on short description prompts
provided by the user in {language}.

46
3851


In [ ]:
all_msg.append(response.content)

In [ ]:
ch_5_pt_3 = """
Part 3
Oh boy. Haven’t the times changed in the island country of Amestris? 15 years ago, women in jeans used to be a rare sight. Tight ones were considered even taboo.
What started as a trend, became a culture. Once a conservative state, now a quintessential fashion hub. All women in Amestris wore jeans most of the time. Except one.
Men have become used to this cultural shift. Except one young man.
"""

response = with_message_history_args_trim.invoke(
    {"messages": [HumanMessage(content=ch_5_pt_3)], "language": "English"},
    config=config,
)

response.content

"**Part 3**\n\nOh boy, the times had certainly changed in the island country of Amestris. 15 years ago, women in jeans were a rare sight, and tight ones were considered even taboo. It was a conservative state, where modesty was valued above all else. But what started as a trend had become a culture. Jeans had become an integral part of the Amestrian wardrobe.\n\nWomen of all ages and shapes wore jeans most of the time, and it was considered the norm. The streets were filled with the sounds of denim and the rustling of fabric as people moved about their daily lives. Amestris had transformed into a quintessential fashion hub, and jeans were at the forefront of it all.\n\nExcept for one.\n\nReynier, the young man who was struggling to adjust to this new reality, was the exception. He was the only one who still found the sight of women in jeans to be shocking and jarring. He couldn't help but feel like he was stuck in a different era, one where modesty was still the norm.\n\nAs he walked t

In [ ]:
print(messages[0].content)
print(messages.__len__())
print(messages[-1].usage_metadata["total_tokens"])

You are a novelist.
Your task is to elaborate a chapter based on short description prompts
provided by the user in {language}.

22
3945


In [ ]:
ch_5_pt_4 = """
Part 4:
Women want to show their bodies off, without having to reveal their skin. The only way to achieve this, is by wearing tight clothing such as jeans.
"""

response = with_message_history_args_trim.invoke(
    {"messages": [HumanMessage(content=ch_5_pt_4)], "language": "English"},
    config=config,
)

response.content

'**Chapter 5: Part 4**\n\nThe sun was setting over the bustling streets of Amestris, casting a warm orange glow over the city. Reynier was walking down the street, lost in thought, when he overheard a group of women chatting about fashion.\n\n"...and the best part is, you can show off your body without having to reveal your skin," one of them said, her voice animated.\n\nReynier\'s ears perked up as he listened in. "What do you mean?" he asked, trying to sound casual.\n\nThe women turned to him, surprised to see a man eavesdropping on their conversation. "Oh, sorry about that," one of them said, laughing. "We were just discussing the advantages of wearing tight clothing, like jeans."\n\nReynier\'s eyes widened as he realized what they were saying. "So, you\'re saying that women wear jeans to show off their bodies without revealing their skin?"\n\nThe women nodded in unison. "Exactly! It\'s all about confidence and self-expression. By wearing tight clothing, we can show off our curves a

In [ ]:
print(messages[0].content)
print(messages.__len__())
print(messages[-1].usage_metadata["total_tokens"])

You are a novelist.
Your task is to elaborate a chapter based on short description prompts
provided by the user in {language}.

26
3862


In [ ]:
for message in store["abc11"].messages:
  print(message.content)
  print(message.response_metadata)

You are a novelist.
Your task is to elaborate a chapter based on short description prompts
provided by the user in {language}.

{}
I'm excited to get started. What's the short description prompt for this chapter? Please go ahead and provide it, and I'll do my best to elaborate a captivating chapter for you.
{'token_usage': {'completion_tokens': 39, 'prompt_tokens': 66, 'total_tokens': 105, 'completion_time': 0.031249449, 'prompt_time': 0.014050874, 'queue_time': None, 'total_time': 0.045300322999999997}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_6a6771ae9c', 'finish_reason': 'stop', 'logprobs': None}


Prologue:

Reynier, a 19 year old boy Reynier  lives with his mother Renuka, a 46 year manager at a bank and his older sister Riya, 22.

His parents separated when he was young. Renuka is also very traditional, only seen in saris, salwars at and Dirndl.

Reynier is a bright student. His creativity rivals that of Da Vinci. He is a college freshman, majoring in fashion desi

In [ ]:
len(messages)

50

In [ ]:
trimmed_messages = trimmer.invoke(messages)

In [ ]:
len(trimmed_messages)

30

### Refs

In [ ]:
###

 - https://python.langchain.com/v0.2/docs/integrations/chat/google_generative_ai/

 - https://python.langchain.com/v0.2/docs/integrations/llms/google_vertex_ai_palm/

 - https://api.python.langchain.com/en/latest/chat_models/langchain_google_genai.chat_models.ChatGoogleGenerativeAI.html#langchain_google_genai.chat_models.ChatGoogleGenerativeAI

 -

## RAG

$ \large \text{RAG is a technique for augmenting LLM knowledge with additional data.} $

LLMs can reason about wide-ranging topics, but their knowledge is limited to the public data up to a specific point in time that they were trained on. If you want to build AI applications that can reason about private data or data introduced after a model's cutoff date, you need to augment the knowledge of the model with the specific information it needs. The process of bringing the appropriate information and inserting it into the model prompt is known as Retrieval Augmented Generation (RAG).

A typical RAG application has two main components:

**Indexing**: a pipeline for ingesting data from a source and indexing it. This usually happens offline.

**Retrieval and generation**: the actual RAG chain, which takes the user query at run time and retrieves the relevant data from the index, then passes that to the model.

### Indexing

 1. **Load**: First we need to load our data. This is done with [Document Loaders](https://python.langchain.com/v0.2/docs/concepts/#document-loaders).

 2. **Split**: [Text splitters](https://python.langchain.com/v0.2/docs/concepts/#text-splitters) break large Documents into smaller chunks. This is useful both for indexing data and for passing it in to a model, since large chunks are harder to search over and won't fit in a model's finite context window.

 At a high level, text splitters work as following:

  - Split the text up into small, semantically meaningful chunks (often sentences).

  - Start combining these small chunks into a larger chunk until you reach a certain size (as measured by some function).

  - Once you reach that size, make that chunk its own piece of text and then start creating a new chunk of text with some overlap (to keep context between chunks).

 3. **Store**: We need somewhere to store and index our splits, so that they can later be searched over. This is often done using a [VectorStore](https://python.langchain.com/v0.2/docs/concepts/#vector-stores) and [Embeddings model](https://python.langchain.com/v0.2/docs/concepts/#embedding-models).

 The Embeddings class is a class designed for interfacing with text embedding models. There are many different embedding model providers (OpenAI, Cohere, Hugging Face, etc) and local models, and this class is designed to provide a standard interface for all of them.

 > The base Embeddings class in LangChain provides two methods: one for embedding documents and one for embedding a query. The former takes as input multiple texts, while the latter takes a single text. The reason for having these as two separate methods is that some embedding providers have different embedding methods for documents (to be searched over) vs queries (the search query itself).

### Retrieval and generation

 4. Retrieve: Given a user input, relevant splits are retrieved from storage using a [Retriever](https://python.langchain.com/v0.2/docs/concepts/#retrievers).

 5. Generate: A ChatModel / LLM produces an answer using a prompt that includes the question and the retrieved data

In [ ]:
!pip install -qU langchain-text-splitters

In [ ]:
!pip install sentence-transformers==2.2.2 --force-reinstall

In [ ]:
!pip install sentence-transformers==2.2.2
!pip install InstructorEmbedding
!pip install langchain_chroma
!pip install -qU langchain-milvus
!pip install --upgrade --quiet pymupdf

In [ ]:
from langchain_community.document_loaders import PyMuPDFLoader

In [ ]:
!wget --max-redirect=20 -O assignment.zip https://www.dropbox.com/scl/fi/0mrhzi6mrwo6ko8xk467d/assignment.zip?rlkey=ektvt12xb2ujtq9ydf4wjfx50&st=iru1ns58&dl=0

--2024-07-23 13:13:47--  https://www.dropbox.com/scl/fi/0mrhzi6mrwo6ko8xk467d/assignment.zip?rlkey=ektvt12xb2ujtq9ydf4wjfx50
Resolving www.dropbox.com (www.dropbox.com)... 162.125.5.18, 2620:100:601f:18::a27d:912
Connecting to www.dropbox.com (www.dropbox.com)|162.125.5.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://uc058dcb0b74556297743c2b578a.dl.dropboxusercontent.com/cd/0/inline/CXTSNLFVYC-tF3efZuokubtCeMYpVIhGX-JI6PeSHBWhY2Mbf_487imavXG38q9Dm6CtLbAvRUQ2BaDD0WJat_1OWcjlYlkirTTWAsChaxPNSDKhKj0Ht9VowYd7c8maw22oWIOOs0w3-yHml6F12JWx/file# [following]
--2024-07-23 13:13:48--  https://uc058dcb0b74556297743c2b578a.dl.dropboxusercontent.com/cd/0/inline/CXTSNLFVYC-tF3efZuokubtCeMYpVIhGX-JI6PeSHBWhY2Mbf_487imavXG38q9Dm6CtLbAvRUQ2BaDD0WJat_1OWcjlYlkirTTWAsChaxPNSDKhKj0Ht9VowYd7c8maw22oWIOOs0w3-yHml6F12JWx/file
Resolving uc058dcb0b74556297743c2b578a.dl.dropboxusercontent.com (uc058dcb0b74556297743c2b578a.dl.dropboxusercontent.com)... 162.125.5.15, 2620

In [ ]:
!unzip ./assignment.zip -d ./assignments

Archive:  ./assignment.zip
  inflating: ./assignments/i1097btc.pdf  
  inflating: ./assignments/i1098c.pdf  
  inflating: ./assignments/i843 1.pdf  
  inflating: ./assignments/i926.pdf  


In [ ]:
!wget --max-redirect=20 -O test.csv https://www.dropbox.com/scl/fi/tr76tmz2n9moxh8lp9wlr/Questions.csv?rlkey=srh7a5l44of1v6l55i01995b6&st=f0er1oyh&dl=0

--2024-07-23 13:13:49--  https://www.dropbox.com/scl/fi/tr76tmz2n9moxh8lp9wlr/Questions.csv?rlkey=srh7a5l44of1v6l55i01995b6
Resolving www.dropbox.com (www.dropbox.com)... 162.125.5.18, 2620:100:601f:18::a27d:912
Connecting to www.dropbox.com (www.dropbox.com)|162.125.5.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://uc70f7e012564cf2aefeaf876047.dl.dropboxusercontent.com/cd/0/inline/CXTq1WqZr0PwWkcUXYD2_SFqGNRiCIuVuio-RQTqRZ5LXS8zYdt1977j5GRgRdrqt7Z2_dsgdZCnnyMDTMNG8e0tHB55Qs4thhoUHjBiUuzNwU9IIJYir-YiHqVZwMveVllrLtKIXOzq_RctnESryweX/file# [following]
--2024-07-23 13:13:50--  https://uc70f7e012564cf2aefeaf876047.dl.dropboxusercontent.com/cd/0/inline/CXTq1WqZr0PwWkcUXYD2_SFqGNRiCIuVuio-RQTqRZ5LXS8zYdt1977j5GRgRdrqt7Z2_dsgdZCnnyMDTMNG8e0tHB55Qs4thhoUHjBiUuzNwU9IIJYir-YiHqVZwMveVllrLtKIXOzq_RctnESryweX/file
Resolving uc70f7e012564cf2aefeaf876047.dl.dropboxusercontent.com (uc70f7e012564cf2aefeaf876047.dl.dropboxusercontent.com)... 162.125.5.15, 2620:

### Loading document

In [ ]:
def read_pdf(path):
  loader = PyMuPDFLoader(
      path
  )
  doc = loader.load()
  return doc

In [ ]:
data = read_pdf("/content/assignments/i1097btc.pdf")
len(data[0].page_content)

5054

In [ ]:
print("First page, starting content: ")
print(data[0].page_content[:500])

First page, starting content: 
Instructions for Form 
1097-BTC
(Rev. December 2019)
Bond Tax Credit
Department of the Treasury
Internal Revenue Service
Section references are to the Internal Revenue Code unless 
otherwise noted.
Future Developments
For the latest information about developments related to 
Form 1097-BTC and its instructions, such as legislation 
enacted after they were published, go to IRS.gov/
Form1097BTC.
Reminders
In addition to these specific instructions, you also should use 
the current General Instructi


In [ ]:
print("Last page, end content: ")
print(data[-1].page_content[4600:])

Last page, end content: 
t amount determined in STEP 1 in 
box 5f. You would carry the same percentage to the Total 
annual credit reported in box 1 for the annual/4th quarter 
reporting period filed with the IRS and sent to the recipient. 
Thus, in Example 2, you would enter in box 1 “27%” (0.27) of 
the amount determined in STEP 1.
Note. For new clean renewable energy bonds issued under 
section 54C and qualified energy conservation bonds issued 
under section 54D, report the credit amount after the 70% 
limit has been applied.
Box 6. Comments
Enter any additional information.
-4-
Instructions for Form 1097-BTC (Rev. 12-2019)



### Splitting into chunks

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

We set `add_start_index=True` so that the character index at which each split Document starts within the initial Document is preserved as metadata attribute “start_index”.

In [ ]:
def split_text(doc):
  text_splitter = RecursiveCharacterTextSplitter(
      chunk_size=1000, chunk_overlap=200, add_start_index=True
  )
  all_splits = text_splitter.split_documents(doc)
  return all_splits


all_splits = split_text(data)
len(all_splits)

30

Note the `start_index`

In [ ]:
all_splits[0].metadata

{'source': '/content/assignments/i1097btc.pdf',
 'file_path': '/content/assignments/i1097btc.pdf',
 'page': 0,
 'total_pages': 4,
 'format': 'PDF 1.7',
 'title': 'Instructions for Form 1097-BTC (Rev. December 2019)',
 'author': 'W:CAR:MP:FP',
 'subject': 'Instructions for Form 1097-BTC, Bond Tax Credit',
 'keywords': '',
 'creator': 'AH XSL Formatter V6.6 MR4 for Linux64 : 6.6.6.37929 (2019/03/11 12:35JST)',
 'producer': 'Antenna House PDF Output Library 6.6.1437 (Linux64); modified using iText 2.1.7 by 1T3XT',
 'creationDate': "D:20191031122810-05'00'",
 'modDate': "D:20191105122303-05'00'",
 'trapped': '',
 'start_index': 0}

In [ ]:
all_splits[-1].metadata

{'source': '/content/assignments/i1097btc.pdf',
 'file_path': '/content/assignments/i1097btc.pdf',
 'page': 3,
 'total_pages': 4,
 'format': 'PDF 1.7',
 'title': 'Instructions for Form 1097-BTC (Rev. December 2019)',
 'author': 'W:CAR:MP:FP',
 'subject': 'Instructions for Form 1097-BTC, Bond Tax Credit',
 'keywords': '',
 'creator': 'AH XSL Formatter V6.6 MR4 for Linux64 : 6.6.6.37929 (2019/03/11 12:35JST)',
 'producer': 'Antenna House PDF Output Library 6.6.1437 (Linux64); modified using iText 2.1.7 by 1T3XT',
 'creationDate': "D:20191031122810-05'00'",
 'modDate': "D:20191105122303-05'00'",
 'trapped': '',
 'start_index': 4816}

In [ ]:
print(all_splits[0].page_content)

Instructions for Form 
1097-BTC
(Rev. December 2019)
Bond Tax Credit
Department of the Treasury
Internal Revenue Service
Section references are to the Internal Revenue Code unless 
otherwise noted.
Future Developments
For the latest information about developments related to 
Form 1097-BTC and its instructions, such as legislation 
enacted after they were published, go to IRS.gov/
Form1097BTC.
Reminders
In addition to these specific instructions, you also should use 
the current General Instructions for Certain Information 
Returns. Those general instructions include information 
about the following topics.
• Who must file.
• When and where to file.
• Electronic reporting.
• Corrected and voided returns.
• Statements to recipients.
• Taxpayer identification numbers (TINs).
• Backup withholding.
• Penalties.
• Other general topics.
You can get the current general instructions from General 
Instructions for Certain Information Returns at IRS.gov/


In [ ]:
print(all_splits[-1].page_content)

Thus, in Example 2, you would enter in box 1 “27%” (0.27) of 
the amount determined in STEP 1.
Note. For new clean renewable energy bonds issued under 
section 54C and qualified energy conservation bonds issued 
under section 54D, report the credit amount after the 70% 
limit has been applied.
Box 6. Comments
Enter any additional information.
-4-
Instructions for Form 1097-BTC (Rev. 12-2019)


In [ ]:
doc_splits = [
    split_text(read_pdf(os.path.join(os.getcwd(), "assignments", _file))) for _file in os.listdir("./assignments")
]

In [ ]:
!rm -rf ./assignments/.ipynb_checkpoints

In [ ]:
len(doc_splits), len(doc_splits[0])

(4, 17)

In [ ]:
all_splits = [split for splits in doc_splits for split in splits]

In [ ]:
all_splits[0]

Document(metadata={'source': '/content/assignments/i1098c.pdf', 'file_path': '/content/assignments/i1098c.pdf', 'page': 0, 'total_pages': 2, 'format': 'PDF 1.7', 'title': 'Instructions for Form 1098-C (Rev. November 2019)', 'author': 'W:CAR:MP:FP', 'subject': 'Instructions for Form 1098-C, Contributions of Motor Vehicles, Boats, and Airplanes', 'keywords': '', 'creator': 'AH XSL Formatter V6.6 MR4 for Linux64 : 6.6.6.37929 (2019/03/11 12:35JST)', 'producer': 'Antenna House PDF Output Library 6.6.1437 (Linux64); modified using iText 2.1.7 by 1T3XT', 'creationDate': "D:20191030112848-05'00'", 'modDate': "D:20191030115658-04'00'", 'trapped': '', 'start_index': 0}, page_content='Instructions for Form 1098-C\n(Rev. November 2019)\nContributions of Motor Vehicles, Boats, and Airplanes\nDepartment of the Treasury\nInternal Revenue Service\nSection references are to the Internal Revenue Code unless otherwise \nnoted.\nFuture Developments\nFor the latest information about developments related t

In [ ]:
all_splits[-1]

Document(metadata={'source': '/content/assignments/i926.pdf', 'file_path': '/content/assignments/i926.pdf', 'page': 7, 'total_pages': 8, 'format': 'PDF 1.7', 'title': 'Instructions for Form 926 (Rev. November 2018)', 'author': 'W:CAR:MP:FP', 'subject': 'Instructions for Form 926, Return by a U.S. Transferor of Property to a Foreign Corporation', 'keywords': '', 'creator': 'AH XSL Formatter V6.5 MR5 for Linux64 : 6.5.8.33214 (2018/05/11 09:14JST)', 'producer': 'Antenna House PDF Output Library 6.5.1276 (Linux64); modified using iText 2.1.7 by 1T3XT', 'creationDate': "D:20181030143532-05'00'", 'modDate': "D:20181113152036-05'00'", 'trapped': '', 'start_index': 2403}, page_content='1 hr., 52 min.\nIf you have comments concerning the accuracy of these time estimates or suggestions for making this form simpler, \nwe would be happy to hear from you. See the instructions for the tax return with which this form is filed.\n-8-\nInstructions for Form 926 (Rev. 11-2018)')

### Create embeddings

In [ ]:
from langchain_community.embeddings import HuggingFaceInstructEmbeddings

In [ ]:
model_name = "hkunlp/instructor-large"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': True}

hf = HuggingFaceInstructEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

/usr/local/lib/python3.10/dist-packages/InstructorEmbedding/instructor.py:7: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'cached_download' (from 'huggingface_hub.file_download') is deprecated and will be removed from version '0.26'. Use `hf_h

.gitattributes:   0%|          | 0.00/1.48k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/270 [00:00<?, ?B/s]

2_Dense/config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/3.15M [00:00<?, ?B/s]

...

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.41k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/461 [00:00<?, ?B/s]

load INSTRUCTOR_Transformer
max_seq_length  512


### Store in vector database

In [ ]:
import pandas as pd

In [ ]:
from langchain_chroma import Chroma

In [ ]:
questions = pd.read_csv("test.csv", index_col="ID")
q_s = questions["Questions"].values

In [ ]:
vectorstore = Chroma.from_documents(documents=all_splits, embedding=hf)

Retriever: An object that returns Documents given a text query

In [ ]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 6})

retrieved_docs = retriever.invoke(q_s[0])

len(retrieved_docs)

6

In [ ]:
print(q_s)

['What is the purpose of Form 843?' 'When should you not use Form 843?'
 'What information should be provided when requesting a refund of excess social security, Medicare, or RRTA tax withheld by one employer?']


In [ ]:
retrieved_docs

### Retrieval

In [ ]:
from langchain_core.output_parsers import StrOutputParser

In [ ]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [ ]:
prompt = ChatPromptTemplate.from_messages(
  [
    (
      "system",
      "You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.",
    ),
    ("user", "Context: {context}\n\nQuestion: {question}")
  ]
)

 - `retriever | format_docs` passes the question through the retriever, generating Document objects, and then to `format_docs` to generate strings;

 - `RunnablePassthrough()` passes through the input question unchanged. It is a Runnable to passthrough inputs unchanged or with additional keys.



In [ ]:
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

In [ ]:
for chunk in rag_chain.stream(q_s[2]):
  print(chunk, end="", flush=True)

When requesting a refund of excess social security, Medicare, or RRTA tax withheld by one employer, you must attach a statement from the employer indicating the amount, if any, the employer has repaid or reimbursed you for excess taxes withheld and the amount, if any, of credit or refund claimed by the employer. If you cannot obtain a statement from the employer, you should attach a statement with the same information to the best of your knowledge and belief, including an explanation of why you could not obtain a statement from the employer.

In [ ]:
response = rag_chain.invoke(q_s[2])
print(response)

When requesting a refund of excess social security, Medicare, or RRTA tax withheld by one employer, you must, if possible, attach a statement from the employer. The statement should indicate the amount, if any, the employer has repaid or reimbursed you for excess taxes withheld, and the amount, if any, of credit or refund claimed by the employer or authorized by you to be claimed by the employer.


In [ ]:
str(q_s[1])

'When should you not use Form 843?'

In [ ]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain

In [ ]:
system_prompt = (
  "You are an assistant for question-answering tasks. "
  "Use the following pieces of retrieved context to answer "
  "the question. If you don't know the answer, say that you "
  "don't know. Use three sentences maximum and keep the "
  "answer concise."
  "\n\n"
  "{context}"
)

prompt = ChatPromptTemplate.from_messages(
  [
    ("system", system_prompt),
    ("human", "{input}"),
  ]
)

 - `create_stuff_documents_chain` specifies how retrieved context is fed into a prompt and LLM. In this case, we will "stuff" the contents into the prompt -- i.e., *we will include all retrieved context without any summarization or other processing.*

 It largely implements our above `rag_chain`, with input keys context and input-- it generates an answer using retrieved context and query.

 - `create_retrieval_chain` adds the retrieval step and propagates the retrieved context through the chain, providing it alongside the final answer. It has input key input, and includes input, context, and answer in its output.

In [ ]:
question_answer_chain = create_stuff_documents_chain(model, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [ ]:
response = rag_chain.invoke({"input": q_s[2]})
print(response["answer"])

When requesting a refund of excess social security, Medicare, or RRTA tax withheld by one employer, you should provide a statement from the employer indicating the following: the amount, if any, the employer has repaid or reimbursed you for excess taxes withheld, and the amount, if any, of credit or refund claimed by the employer or authorized by you to be claimed by the employer. If you cannot obtain a statement from the employer, you should attach a statement with the same information to the best of your knowledge and belief, and include an explanation of why you could not obtain a statement from the employer.


#### Returning sources

In [ ]:
for document in response["context"]:
  print(document)
  print()

page_content='interest rate of zero under section 6621(d). See Requesting Net 
Interest Rate of Zero on Overlapping Tax Underpayments and 
Overpayments, later, for more information.
Refund of excess social security, Medicare, or RRTA tax. If 
you are claiming a refund of excess social security, Medicare, or 
RRTA tax withheld by one employer, you must, if possible, 
attach a statement from the employer. The statement should 
indicate the following.
• The amount, if any, the employer has repaid or reimbursed 
you for excess taxes withheld.
• The amount, if any, of credit or refund claimed by the 
employer or authorized by you to be claimed by the employer.
The employer should include in the statement the fact that it is 
made in support of your claim for refund of employee tax paid by 
the employer to the IRS.
If you cannot obtain a statement from the employer, you 
should attach a statement with the same information to the best 
CAUTION
!
-4-
Instructions for Form 843 (Rev. December 20